In [ ]:
#NBA Championshi Prediction
#01 - Data Collection Notebook

In [ ]:
import pandas as pd
import time 
from nba_api.stats.endpoints import leaguegamelog, leaguedashteamstats

In [2]:
#Helper Function
def save_df(df, path):
    df.to_csv(path, index=False)
    print(f"Saved file to {path}")

In [ ]:
#1. Pull Regular Season Games

In [4]:
print("Pulling regular season game logs...")
log = leaguegamelog.LeagueGameLog(season='2024-25', season_type_all_star='Regular Season')
games_df = log.get_data_frames()[0]
save_df(games_df, '../data/raw/2024-25_regular_season_games.csv')
time.sleep(1) #API care
print("2024-25_regular_season_games Saved")

Pulling regular season game logs...
Saved file to ../data/raw/2024-25_regular_season_games.csv
2024-25_regular_season_games Saved


In [ ]:
#2. Pull Team Base Stats

In [5]:
print("Pulling team base stats...")
team_base = leaguedashteamstats.LeagueDashTeamStats(
    season='2024-25',
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Base'
)
teams_base_df = team_base.get_data_frames()[0]
save_df(teams_base_df, '../data/raw/2024-25_teams_stats_base.csv')
time.sleep(1)
print("2024-25_team_stats_base Saved")

Pulling team base stats...
Saved file to ../data/raw/2024-25_teams_stats_base.csv
2024-25_team_stats_base Saved


In [ ]:
#3. Pull Teams Advanced Stats

In [6]:
print("Pulling team advanced stats...")
team_advanced = leaguedashteamstats.LeagueDashTeamStats(
    season='2024-25',
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Advanced' 
)
teams_advanced_df = team_advanced.get_data_frames()[0]
save_df(teams_advanced_df, '../data/raw/2024-25_teams_stats_advanced.csv')
time.sleep(1)
print("2024-25_team_stats_advanced Saved")

Pulling team advanced stats...
Saved file to ../data/raw/2024-25_teams_stats_advanced.csv
2024-25_team_stats_advanced Saved
